## Частичное обучение

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
import pandas as pd
from sklearn.preprocessing import FunctionTransformer

def learn_clf(model, trainDf, dense):
    """Modify data to tf-idf and learn a model"""
    
    if dense:
        text_clf = Pipeline([('tfidfvect', TfidfVectorizer(ngram_range=(1,2))),
                             ('vect_dense', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)),
                             ('clf', model),
                            ])
    else:
        text_clf = Pipeline([('tfidfvect', TfidfVectorizer(ngram_range=(1,2))),
                             ('clf', model),
                            ])
    
    text_clf.fit(trainDf.text, trainDf.label)
    
    return text_clf

In [2]:
import numpy as np

def semi_learn(classesList, predicted_proba, trainDf, testDf, t):
    """Expand training data"""
           
    startIndex = len(trainDf)
    
    for index, result in enumerate(predicted_proba):
        if max(result) >= t:
            label = np.argmax(result)
            trainDf = trainDf.append({'label': classesList[label],
                                      'text': testDf.iloc[index].text},
                                     ignore_index=True)

    endIndex = len(trainDf)
    
    lenBefore = len(testDf)
    
    # Remove expanded data from non-labeled data
    
    testDf = testDf.loc[~testDf['text'].isin(trainDf.text)].reset_index(drop=True)

    lenAfter = len(testDf)
    
    # How many expanded per step
    count = lenBefore - lenAfter
    
    return trainDf, testDf, count

In [7]:
import os
from collections import Counter
import sys
import unidecode

def result_func(dataset, mode, wordList, labeledPath, nonlabeledPath, savePath, model, dense=False, t=1.0):

    for word in wordList:
#         print(word)
        bool_break = False
        
        labeledWord = labeledPath + unidecode.unidecode(word) + '.csv'
        labeledDf = pd.read_csv(labeledWord, header=None, sep='\t', engine='python', encoding='utf-8')
        labeledDf = labeledDf.rename(columns={labeledDf.columns[0]: "label", 
                                              labeledDf.columns[1]: "text"}
                                    )        
        try:
            nonlabeledWord = nonlabeledPath + word + '.csv'
            nonLabeledDf = pd.read_csv(nonlabeledWord, header=None, sep='\t', engine='python', encoding='utf-8')
            nonLabeledDf = nonLabeledDf.rename(columns={nonLabeledDf.columns[0]: "text"})
        except Exception as e:
            if hasattr(e, 'message'):
                print(e.message)
            else:
                print(e)
            nonLabeledDf = pd.DataFrame()
#             continue
        
        print(word, len(labeledDf), Counter(labeledDf.label))

        count = -1
        while (count != 0 and len(nonLabeledDf) > 0):
            try:
                clf = learn_clf(model, labeledDf, dense)
                predicted = clf.predict_proba(nonLabeledDf.text)
                labeledDf, nonLabeledDf, count = semi_learn(clf.classes_, predicted, 
                                                            labeledDf, nonLabeledDf, t)
                
                print('test', len(labeledDf), len(nonLabeledDf), count)
#                 sys.exit()
            except Exception as e:
                print('Error', word)

                if hasattr(e, 'message'):
                    print(e.message)
                else:
                    print(e)

                bool_break = True
                break
                
        if bool_break == False:      
            if not os.path.exists(savePath + 'Not Expanded/'):
                os.makedirs(savePath + 'Not Expanded/')

            outputName = savePath + unidecode.unidecode(word) + '.csv'
            labeledDf.to_csv(outputName, sep='\t', header=False, 
                             index=False, encoding='utf-8')
            
            LeftName = savePath + '/Not Expanded/' + unidecode.unidecode(word) + '.csv'
            nonLabeledDf.to_csv(LeftName, sep='\t', header=False,
                                index=False, encoding='utf-8')
            
            print('Became', len(labeledDf))
            print('Left', len(nonLabeledDf))
            print('-----')

    print('Finish')

# Начало работы

## Файл train

In [4]:
dataset = 'bts-rnc'

wordList = ['балка', 'вид', 'винт', 'горн', 'губа', 'жаба', 'клетка',
            'крыло', 'купюра', 'курица', 'лавка', 'лайка', 'лев', 'лира',
            'мина', 'мишень', 'обед', 'оклад', 'опушка', 'полис', 'пост', 
            'поток', 'проказа', 'пропасть', 'проспект', 'пытка', 'рысь',
            'среда', 'хвост', 'штамп'
           ]
mode = 'train'

### 1.1. kNN

In [5]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(weights='uniform')
model_name = str(model.__class__.__name__)

In [6]:
labeledPath = "Data/Annotated/lemma/{}({})/".format(dataset, mode)
nonlabeledPath = "Data/Non-Annotated/lemma/{}({})/".format(dataset, mode)
savePath = "Data/Expanded/{}({})/{}/".format(dataset, mode, model_name)

In [8]:
%%time
result_func(dataset=dataset, mode=mode, wordList=wordList, 
            labeledPath=labeledPath, nonlabeledPath=nonlabeledPath,
            savePath=savePath, model=model, t=1.00)

балка 91 Counter({34297: 47, 39329: 44})
Became 1471
Left 606
-----
вид 105 Counter({18983: 35, 38473: 34, 46934: 21, 16525: 15})
Became 1226
Left 1618
-----
винт 84 Counter({16398: 32, 17916: 22, 39939: 22, 32507: 8})
Became 757
Left 1170
-----
горн 100 Counter({30349: 40, 1: 31, 32374: 24, 0: 5})
Became 1243
Left 376
-----
губа 73 Counter({36563: 50, 32217: 17, 40001: 6})
Became 3289
Left 46
-----
жаба 94 Counter({38123: 29, 1: 28, 0: 19, 22390: 18})
Became 492
Left 644
-----
клетка 151 Counter({16575: 33, 15531: 32, 24810: 29, 19864: 29, 15764: 28})
Became 2295
Left 709
-----
крыло 166 Counter({35257: 37, 31524: 33, 19801: 28, 36977: 24, 20531: 22, 0: 12, 29548: 10})
Became 1188
Left 1031
-----
купюра 88 Counter({25844: 54, 12767: 34})
Became 1318
Left 225
-----
курица 60 Counter({38375: 33, 33131: 27})
Became 1523
Left 328
-----
лавка 52 Counter({19335: 27, 38881: 25})
Became 1416
Left 738
-----
лайка 66 Counter({21481: 45, 30243: 21})
Became 322
Left 90
-----
лев 68 Counter({28022

### 1.2. SVM RBF

In [5]:
from sklearn.svm import SVC

model = SVC(gamma=2, C=1, probability=True)
model_name = 'RBF-SVM'

In [6]:
labeledPath = "Data/Annotated/lemma/{}({})/".format(dataset, mode)
nonlabeledPath = "Data/Non-Annotated/lemma/{}({})/".format(dataset, mode)
savePath = "Data/Expanded/{}({})/{}/".format(dataset, mode, model_name)

In [ ]:
%%time

wordList2 = ['вид', 'винт', 'горн', 'губа', 'жаба', 'клетка',
            'крыло', 'купюра', 'курица', 'лавка', 'лайка', 'лев', 'лира',
            'мина', 'мишень', 'обед', 'оклад', 'опушка', 'полис', 'пост', 
            'поток', 'проказа', 'пропасть', 'проспект', 'пытка', 'рысь',
            'среда', 'хвост', 'штамп'
           ]

result_func(dataset=dataset, mode=mode, wordList=wordList2,
            labeledPath=labeledPath, nonlabeledPath=nonlabeledPath,
            savePath=savePath, model=model, dense=True, t=0.60)

вид 105 Counter({18983: 35, 38473: 34, 46934: 21, 16525: 15})
Became 109
Left 2735
-----
винт 84 Counter({16398: 32, 17916: 22, 39939: 22, 32507: 8})
Became 93
Left 1834
-----
горн 100 Counter({30349: 40, 1: 31, 32374: 24, 0: 5})


### 1.3. Gaussian

In [5]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

model = GaussianProcessClassifier(1.0 * RBF(1.0), n_jobs=2)
model_name = str(model.__class__.__name__)

In [6]:
labeledPath = "Data/Annotated/lemma/{}({})/".format(dataset, mode)
nonlabeledPath = "Data/Non-Annotated/lemma/{}({})/".format(dataset, mode)
savePath = "Data/Expanded/{}({})/{}/".format(dataset, mode, model_name)

In [ ]:
%%time

result_func(dataset=dataset, mode=mode, wordList=wordList,
            labeledPath=labeledPath, nonlabeledPath=nonlabeledPath,
            savePath=savePath, model=model, dense=True, t=0.60)

балка 91 Counter({34297: 47, 39329: 44})
test 91 1986 0
Became 91
Left 1986
-----
вид 105 Counter({18983: 35, 38473: 34, 46934: 21, 16525: 15})
test 139 2705 34
test 210 2634 71
test 330 2514 120
test 485 2359 155
test 1259 1585 774


### 2.1. Semi-supervied RBF

In [31]:
from sklearn.semi_supervised import LabelPropagation
model = LabelPropagation(kernel='rbf', gamma=15, n_jobs=2)
model_name = 'KNeighborsClassifier_RBF'

In [30]:
labeledPath = "Data/Expanded/{}({})/KNeighborsClassifier/".format(dataset, mode)
nonlabeledPath = "Data/Expanded/{}({})/KNeighborsClassifier/Not Expanded/".format(dataset, mode)
savePath = "Data/Expanded/{}({})/{}/".format(dataset, mode, model_name)

In [40]:
%%time
result_func(dataset=dataset, mode=mode, wordList=wordList, 
            labeledPath=labeledPath, nonlabeledPath=nonlabeledPath,
            savePath=savePath, model=model, dense=True, t=0.95)

балка 1471 Counter({39329: 1280, 34297: 191})
Became 1528
Left 549
-----
вид 1226 Counter({38473: 537, 18983: 495, 46934: 172, 16525: 22})
Became 1310
Left 1534
-----
винт 757 Counter({39939: 360, 17916: 242, 16398: 147, 32507: 8})
Became 826
Left 1101
-----
горн 1243 Counter({30349: 1173, 1: 39, 32374: 26, 0: 5})
Became 1375
Left 244
-----
губа 3289 Counter({36563: 3259, 32217: 24, 40001: 6})
Became 3320
Left 15
-----
жаба 492 Counter({38123: 407, 22390: 32, 1: 28, 0: 25})
Became 595
Left 541
-----
клетка 2295 Counter({15531: 2061, 19864: 101, 24810: 59, 16575: 39, 15764: 35})
Became 2363
Left 641
-----
крыло 1188 Counter({19801: 956, 35257: 82, 20531: 66, 31524: 38, 36977: 24, 0: 12, 29548: 10})
Became 1271
Left 948
-----
купюра 1318 Counter({25844: 1246, 12767: 72})
Became 1360
Left 183
-----
курица 1523 Counter({33131: 1402, 38375: 121})
Became 1566
Left 285
-----
лавка 1416 Counter({38881: 1158, 19335: 258})
Became 1478
Left 676
-----
лайка 322 Counter({21481: 299, 30243: 23})
Bec

### 2.2. SVM RBF

In [11]:
from sklearn.svm import SVC

model = SVC(gamma=2, C=1, probability=True)
model_name = 'KNeighborsClassifier_RBF-SVM'

In [12]:
labeledPath = "Data/Expanded/{}({})/KNeighborsClassifier/".format(dataset, mode)
nonlabeledPath = "Data/Expanded/{}({})/KNeighborsClassifier/Not Expanded/".format(dataset, mode)
savePath = "Data/Expanded/{}({})/{}/".format(dataset, mode, model_name)

In [ ]:
%%time
result_func(dataset=dataset, mode=mode, wordList=wordList, 
            labeledPath=labeledPath, nonlabeledPath=nonlabeledPath,
            savePath=savePath, model=model, dense=True, t=0.95)

балка 1471 Counter({39329: 1280, 34297: 191})


### 2.3. Gaussian

In [8]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

model = GaussianProcessClassifier(1.0 * RBF(1.0))
model_name = 'KNeighborsClassifier_GaussianProcessClassifier'

In [6]:
labeledPath = "Data/Expanded/{}({})/KNeighborsClassifier/".format(dataset, mode)
nonlabeledPath = "Data/Expanded/{}({})/KNeighborsClassifier/Not Expanded/".format(dataset, mode)
savePath = "Data/Expanded/{}({})/{}/".format(dataset, mode, model_name)

In [7]:
%%time
result_func(dataset=dataset, mode=mode, wordList=wordList, 
            labeledPath=labeledPath, nonlabeledPath=nonlabeledPath,
            savePath=savePath, model=model, dense=True, t=0.95)

балка 91 Counter({34297: 47, 39329: 44})
Became 91
Left 1986
-----
вид 105 Counter({18983: 35, 38473: 34, 46934: 21, 16525: 15})
Became 105
Left 2739
-----
винт 84 Counter({16398: 32, 17916: 22, 39939: 22, 32507: 8})
Became 84
Left 1843
-----
горн 100 Counter({30349: 40, 1: 31, 32374: 24, 0: 5})
Became 100
Left 1519
-----
губа 73 Counter({36563: 50, 32217: 17, 40001: 6})
Became 73
Left 3262
-----
жаба 94 Counter({38123: 29, 1: 28, 0: 19, 22390: 18})
Became 94
Left 1042
-----
клетка 151 Counter({16575: 33, 15531: 32, 24810: 29, 19864: 29, 15764: 28})


KeyboardInterrupt: 

## Файл test

In [4]:
dataset = 'bts-rnc'

wordList = ['акция', 'баба', 'байка', 'бум', 'бычок', 'вал', 'газ', 'гвоздика',
             'гипербола', 'град', 'гусеница', 'дождь', 'домино', 'забой', 'икра',
             'кабачок', 'капот', 'карьер', 'кличка', 'ключ', 'кок', 'кольцо',
             'концерт', 'котелок', 'крона', 'круп', 'кулак', 'лейка', 'лук',
             'мандарин', 'ножка', 'опора', 'патрон', 'печать', 'пол', 'полоз',
             'почерк', 'пробка', 'рак', 'рок', 'свет', 'секрет', 'скат', 'слог',
             'стан', 'стопка', 'таз', 'такса', 'тюрьма', 'шах', 'шашка'
            ]

mode = 'test'

In [5]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(weights='uniform')

model_name = str(model.__class__.__name__)

In [6]:
labeledPath = "Data/Annotated/lemma/{}({})/".format(dataset, mode)
nonlabeledPath = "Data/Non-Annotated/lemma/{}({})/".format(dataset, mode)
savePath = "Data/Expanded/{}({})/{}/".format(dataset, mode, model_name)

In [7]:
%%time
result_func(dataset=dataset, mode=mode, wordList=wordList, 
            labeledPath=labeledPath, nonlabeledPath=nonlabeledPath,
            savePath=savePath, model=model, t=1.00)

акция 134 Counter({29853: 69, 28176: 38, 15738: 27})
Became 1500
Left 798
-----
баба 111 Counter({29090: 42, 38405: 24, 21130: 23, 1: 13, 0: 9})
Became 2352
Left 760
-----
байка 79 Counter({16141: 75, 39858: 4})
Became 866
Left 11
-----
бум 56 Counter({41843: 33, 32940: 19, 18362: 4})
Became 831
Left 179
-----
бычок 66 Counter({27009: 34, 0: 28, 26270: 4})
Became 257
Left 177
-----
вал 109 Counter({33648: 34, 0: 27, 41024: 25, 1: 23})
Became 1047
Left 650
-----
газ 61 Counter({23414: 28, 17569: 21, 16756: 12})
Became 2186
Left 355
-----
гвоздика 75 Counter({31219: 35, 0: 22, 26662: 18})
Became 1041
Left 491
-----
гипербола 38 Counter({1: 24, 0: 14})
Became 234
Left 48
-----
град 82 Counter({13861: 30, 29527: 21, 0: 16, 35134: 15})
Became 1421
Left 260
-----
гусеница 43 Counter({19345: 24, 21860: 19})
Became 957
Left 623
-----
дождь 56 Counter({40011: 35, 22422: 21})
Became 2294
Left 119
-----
домино 91 Counter({38622: 37, 13675: 30, 1: 24})
Became 566
Left 271
-----
забой 44 Counter({3

In [8]:
from sklearn.semi_supervised import LabelPropagation
model = LabelPropagation(kernel='rbf', gamma=15, n_jobs=2)
model_name = 'KNeighborsClassifier_RBF'

In [9]:
labeledPath = "Data/Expanded/{}({})/KNeighborsClassifier/".format(dataset, mode)
nonlabeledPath = "Data/Expanded/{}({})/KNeighborsClassifier/Not Expanded/".format(dataset, mode)
savePath = "Data/Expanded/{}({})/{}/".format(dataset, mode, model_name)

In [22]:
%%time
result_func(dataset=dataset, mode=mode, wordList=wordList, 
            labeledPath=labeledPath, nonlabeledPath=nonlabeledPath,
            savePath=savePath, model=model, dense=True, t=0.95)

акция 1500 Counter({29853: 1251, 28176: 217, 15738: 32})
Became 1568
Left 730
-----
баба 2352 Counter({29090: 2167, 21130: 130, 38405: 30, 1: 16, 0: 9})
Became 2487
Left 625
-----
байка 866 Counter({16141: 862, 39858: 4})
Became 876
Left 1
-----
бум 831 Counter({41843: 801, 32940: 26, 18362: 4})
Became 895
Left 115
-----
бычок 257 Counter({27009: 201, 0: 52, 26270: 4})
Became 285
Left 149
-----
вал 1047 Counter({33648: 869, 41024: 82, 0: 73, 1: 23})
Became 1123
Left 574
-----
газ 2186 Counter({17569: 2069, 23414: 105, 16756: 12})
Became 2265
Left 276
-----
гвоздика 1041 Counter({31219: 823, 26662: 136, 0: 82})
Became 1065
Left 467
-----
гипербола 234 Counter({1: 220, 0: 14})
Became 239
Left 43
-----
град 1421 Counter({29527: 1309, 0: 49, 13861: 40, 35134: 23})
Became 1441
Left 240
-----
гусеница 957 Counter({21860: 541, 19345: 416})
Became 998
Left 582
-----
дождь 2294 Counter({40011: 2266, 22422: 28})
Became 2383
Left 30
-----
домино 566 Counter({38622: 468, 1: 65, 13675: 33})
Became 